In [14]:
# Import required libraries
from google.colab import userdata
import os
import gradio as gr

# Set environment variables
os.environ['groc_key'] = userdata.get('groc_key')
os.environ['LANGSMITH_API_KEY'] = userdata.get('smith')
os.environ['LANGCHAIN_TRACING_V2'] = 'true'
os.environ['LANGCHAIN_PROJECT'] = 'langgraph_project'
groc_key = os.environ.get('groc_key')

# Import ChatGroq and other dependencies
from langchain_groq import ChatGroq
llm = ChatGroq(groq_api_key=groc_key, model_name='Gemma2-9b-It')

from typing import Annotated
from typing_extensions import TypedDict
from langgraph.graph import StateGraph, START, END
from langgraph.graph.message import add_messages

# Define the state and chatbot function
class State(TypedDict):
  messages: Annotated[list, add_messages]

graph_builder = StateGraph(State)

def chatbot(state: State):
  return {"messages": llm.invoke(state["messages"])}

# Build the graph
graph_builder.add_node("chatbot", chatbot)
graph_builder.add_edge(START, "chatbot")
graph_builder.add_edge("chatbot", END)
graph = graph_builder.compile()

# Chatbot interaction function
def chat(user_input, history):
    history = history or []
    # Append the user input to the conversation
    history.append(("user", user_input))

    # Display the "Generating response..." message in the assistant's slot
    history.append(("assistant", "Generating response..."))

    # Stream the response from the graph
    for event in graph.stream({"messages": ("user", user_input)}):
        for value in event.values():
            # Replace the placeholder response with the actual assistant response
            history[-1] = ("assistant", value["messages"].content)
            return history

# Gradio Interface - App Layout
with gr.Blocks(theme=gr.themes.Monochrome()) as demo:
    # Header of the app
    gr.Markdown("<h1 style='text-align: center;'>Chatbot App</h1>")

    with gr.Row():
        # Chatbot and input box in a vertical stack
        with gr.Column(scale=8):
            chatbot_output = gr.Chatbot(label="Chatbot Conversation", height=500)
            user_input = gr.Textbox(placeholder="Type your message here...", label="Your Input", show_label=False)

        # Sidebar for extra options (future extensions)
        with gr.Column(scale=2):
            gr.Markdown("## Chatbot Info")
            gr.Markdown("This chatbot is powered by LangChain and Groq models. You can ask questions or engage in conversation.")

    # Store conversation history
    state = gr.State([])  # For storing conversation history

    # Function to handle responses
    def respond(user_input, history):
        history = chat(user_input, history)
        return history, history  # Updating both chatbot and state

    # When user submits, update the chat and history
    user_input.submit(respond, [user_input, state], [chatbot_output, state])

# Launch the interface
demo.launch()


Setting queue=True in a Colab notebook requires sharing enabled. Setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
Running on public URL: https://afbc0c92dc70b02275.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)
